In [1]:
!pip install faker pandas numpy
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()
np.random.seed(42)

def generate_final_dataset(n_users=500, snapshots_per_goal=3):
    categories = ['retirement', 'housing', 'education', 'vehicle', 'travel', 'business']
    timelines = [6, 12, 24, 36, 60, 120, 240]  # Include more short timelines

    users = []
    for user_id in range(1, n_users + 1):
        annual_income = round(np.random.lognormal(10.5, 0.4), 2)
        total_savings = round(np.random.weibull(1.5) * min(annual_income * 2, 100000), 2)
        total_debt = round(np.random.exponential(scale=annual_income * 0.5), 2)
        users.append({
            'user_id': f"U{user_id:04d}",
            'age': np.random.randint(18, 70),
            'annual_income': annual_income,
            'total_savings': total_savings,
            'total_debt': total_debt,
            'debt_to_income_ratio': round(total_debt / annual_income, 2) if annual_income > 0 else 0,
            'risk_tolerance_score': round(np.random.beta(2, 3), 2),
            'credit_score': np.random.randint(300, 850)
        })
    users_df = pd.DataFrame(users)

    data = []
    goal_id = 1
    snapshot_id = 1

    for _, user in users_df.iterrows():
        num_goals = np.random.randint(3, 8)
        for _ in range(num_goals):
            category = np.random.choice(categories)
            target_amount = round(np.random.lognormal(np.log(50000), 0.5), 2)
            timeline_months = np.random.choice(timelines)
            monthly_income = user['annual_income'] / 12
            # Allow higher contributions for short timelines
            max_contribution = min(monthly_income * 0.75, target_amount / timeline_months * 2)
            monthly_contribution = round(np.random.uniform(0, max_contribution), 2)

            for _ in range(snapshots_per_goal):
                snapshot_date = fake.date_between(start_date='-2y', end_date='today')
                current_progress = round(np.random.uniform(0, target_amount * 1.5), 2)  # Allow full progress
                inflation_rate = round(np.random.uniform(1.0, 9.0), 1)
                interest_rate = round(np.random.uniform(1.5, 12.0), 2)
                market_sentiment = round(np.random.beta(2, 3), 2)
                # Updated priority score with enhanced weights and no cap
                contribution_factor = monthly_contribution / (target_amount / timeline_months) if timeline_months > 0 else 0
                feasibility_factor = current_progress / target_amount if target_amount > 0 else 0
                priority_score = round(min(1.0, 0.4 + (contribution_factor * 0.5 + feasibility_factor * 0.4) + np.random.normal(0, 0.05)), 2)

                progress_ratio = current_progress / target_amount if target_amount > 0 else 0
                contribution_to_feasibility = monthly_contribution / (target_amount / timeline_months) if timeline_months > 0 else 0

                data.append({
                    'user_id': user['user_id'],
                    'goal_id': f"G{goal_id:05d}",
                    'snapshot_id': f"S{snapshot_id:06d}",
                    'snapshot_date': snapshot_date.strftime('%Y-%m-%d'),
                    'age': user['age'],
                    'annual_income': user['annual_income'],
                    'total_savings': user['total_savings'],
                    'total_debt': user['total_debt'],
                    'debt_to_income_ratio': user['debt_to_income_ratio'],
                    'risk_tolerance_score': user['risk_tolerance_score'],
                    'credit_score': user['credit_score'],
                    'goal_category': category,
                    'target_amount': target_amount,
                    'current_progress': current_progress,
                    'timeline_months': timeline_months,
                    'goal_feasibility': round(target_amount / timeline_months, 2) if timeline_months > 0 else 0,
                    'monthly_contribution': monthly_contribution,
                    'progress_ratio': progress_ratio,
                    'contribution_to_feasibility': contribution_to_feasibility,
                    'inflation_rate': inflation_rate,
                    'interest_rate': interest_rate,
                    'market_sentiment': market_sentiment,
                    'priority_score': priority_score
                })
                snapshot_id += 1
            goal_id += 1

    df = pd.DataFrame(data)
    df = pd.get_dummies(df, columns=['goal_category'], prefix='category')
    df = df.sort_values(['user_id', 'goal_id', 'snapshot_date'])

    return df

final_df = generate_final_dataset(n_users=500, snapshots_per_goal=3)
final_df.to_csv('final_financial_dataset.csv', index=False)

print("=== Final Dataset Preview ===")
print(final_df.head())
print("\n=== Column Info ===")
print(final_df.dtypes)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.3 MB/s eta 0:00:00
=== Final Dataset Preview ===
  user_id goal_id snapshot_id snapshot_date  age  annual_income  \
2   U0001  G00001     S000003    2024-08-24   56       44297.59   
1   U0001  G00001     S000002    2025-06-22   56       44297.59   
0   U0001  G00001     S000001    2025-07-09   56       44297.59   
5   U0001  G00002     S000006    2024-12-19   56       44297.59   
4   U0001  G00002     S000005    2025-03-12   56       44297.59   

   total_savings  total_debt  debt_to_income_ratio  risk_tolerance_score  ...  \
2      106433.42    20220.58                  0.46                  0.37  ...   
1      106433.42    20220.58                  0.46                  0.37  ...   
0      106433.42    20220.58                  0.46                  0.37  ...   
5      106433.42    20220.58                  0.46                  0.37  ...   
4      106433.42    20220.58                  0.46                  0.37  ...   

   

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pickle
import time

# Start timer
start_time = time.time()

# Load dataset
df = pd.read_csv('/content/final_financial_dataset.csv')

# Verify all expected features are present
features = [col for col in df.columns if col not in ['user_id', 'goal_id', 'snapshot_id', 'snapshot_date', 'priority_score', 'goal_category']]
assert all(col in df.columns for col in features), "Missing features in DataFrame"

# Scale features
scaler = StandardScaler()
X = df[features]
X_scaled = scaler.fit_transform(X)
y = df['priority_score']

# Split data
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6]
}

# Hyperparameter tuning
grid_search = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)
model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Cross-validated R² score
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='r2')
print("Cross-validated R² Scores:", cv_scores)
print("Mean Cross-validated R²:", cv_scores.mean())
print("Standard Deviation of R²:", cv_scores.std())

# Evaluate on test set
y_pred = model.predict(X_test_scaled)
print("Test R² Score:", r2_score(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred))

# Save model and scaler
with open('gbr_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Prioritization function
def prioritize_goals(df, model, scaler):
    # Check for varying features across snapshots
    group_cols = ['user_id', 'goal_id']
    for col in features:
        if col in df.columns and df.groupby(group_cols)[col].nunique().max() > 1:
            print(f"Warning: {col} varies across snapshots")

    # Scale input features
    X = df[features]
    X_scaled = scaler.transform(X)
    df['predicted_priority'] = model.predict(X_scaled)

    # Aggregate snapshots
    agg_dict = {'predicted_priority': 'mean'}
    for col in features:
        if col in df.columns:
            agg_dict[col] = 'first'
    prioritized_goals = df.groupby(group_cols).agg(agg_dict).reset_index()

    # Category assignment
    category_cols = [f'category_{cat}' for cat in ['retirement', 'housing', 'education', 'vehicle', 'travel', 'business']]
    def get_category(row):
        return max([(col, row[col]) for col in category_cols if col in row], key=lambda x: x[1])[0].replace('category_', '')
    prioritized_goals['category'] = prioritized_goals.apply(get_category, axis=1)

    # Calculate adjusted priority (synced with app)
    # prioritized_goals['adjusted_priority'] = prioritized_goals.apply(
    #     lambda row: min(1.0, row['predicted_priority'] * (1 + (row['monthly_contribution'] / (row['target_amount'] / row['timeline_months'])) * 2 + (1 / row['timeline_months']) * 5)
    #     if row['target_amount'] > 0 and row['timeline_months'] > 0 else row['predicted_priority']), axis=1)

    prioritized_goals['adjusted_priority'] = prioritized_goals.apply(
    lambda row: min(1.0, row['predicted_priority'] * (1 + min(1.0, row['monthly_contribution'] / (row['target_amount'] / row['timeline_months'])) * 1 + (1 / row['timeline_months']) * 2) * max(0.5, 1 - (row['target_amount'] / (row['monthly_contribution'] * row['timeline_months'] if row['monthly_contribution'] * row['timeline_months'] > 0 else 1))))
    if row['target_amount'] > 0 and row['timeline_months'] > 0 else row['predicted_priority'], axis=1)

    # Sort the prioritized goals
    prioritized_goals = prioritized_goals.sort_values(['user_id', 'adjusted_priority'], ascending=[True, False])

    # Define the action recommendation function
    # def recommend_action(row):
    #     if row['adjusted_priority'] > 0.7:
    #         return "Focus on this goal"
    #     elif row['adjusted_priority'] > 0.4:
    #         return "Maintain steady progress"
    #     else:
    #         return "Consider delaying or adjusting"

    def recommend_action(row):
        feasibility_ratio = row['monthly_contribution'] * row['timeline_months'] / row['target_amount'] if row['target_amount'] > 0 else 0
        if row['adjusted_priority'] > 0.7 and feasibility_ratio > 0.5:
            return "Focus on this goal"
        elif row['adjusted_priority'] > 0.3 and feasibility_ratio > 0.1:
            return "Maintain steady progress"
        else:
            return "Consider delaying or adjusting"

    # Apply the recommendation function
    prioritized_goals['recommended_action'] = prioritized_goals.apply(recommend_action, axis=1)

    return prioritized_goals

# Generate prioritized goals
prioritized_df = prioritize_goals(df, model, scaler)
prioritized_df.to_csv('prioritized_goals.csv', index=False)

# Preview for a sample user
sample_user = prioritized_df[prioritized_df['user_id'] == 'U0001']
print("\n=== Prioritized Goals for User U0001 ===")
print(sample_user[['user_id', 'goal_id', 'category', 'target_amount', 'timeline_months', 'monthly_contribution', 'adjusted_priority', 'recommended_action']])

# End timer
print(f"\nTotal runtime: {time.time() - start_time:.2f} seconds")

Best parameters: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Cross-validated R² Scores: [0.95051914 0.95021465 0.94600221 0.94977016 0.94626117]
Mean Cross-validated R²: 0.9485534640850138
Standard Deviation of R²: 0.0019933522037420045
Test R² Score: 0.950594601844232
Test MAE: 0.021363612079905312

=== Prioritized Goals for User U0001 ===
  user_id goal_id    category  target_amount  timeline_months  \
3   U0001  G00004  retirement       69701.15              240   
1   U0001  G00002  retirement       21194.72               24   
4   U0001  G00005     vehicle       94973.05              120   
2   U0001  G00003      travel       26222.78               60   
0   U0001  G00001     vehicle       82496.62              120   
5   U0001  G00006      travel       46357.05               24   

   monthly_contribution  adjusted_priority        recommended_action  
3                552.28           1.000000        Focus on this goal  
1                575.58           0.865632

In [3]:
!pip install streamlit streamlit-echarts pandas numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [4]:
!pip install matplotlib

In [17]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from streamlit_echarts import st_echarts


# Load model
with open('/content/gbr_model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('/content/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Define expected feature names as per the model
expected_features = [
    'age', 'annual_income', 'total_savings', 'total_debt', 'debt_to_income_ratio',
    'risk_tolerance_score', 'credit_score', 'target_amount', 'current_progress',
    'timeline_months', 'goal_feasibility', 'monthly_contribution',
    'progress_ratio', 'contribution_to_feasibility', 'inflation_rate',
    'interest_rate', 'market_sentiment', 'category_business',
    'category_education', 'category_housing', 'category_retirement',
    'category_travel', 'category_vehicle'
]

# Streamlit app
st.title("AI-Driven Financial Goal Prioritization")

# User input
st.header("Enter Your Financial Details")
age = st.slider("Age", 18, 70, 30)
annual_income = st.number_input("Annual Income ($)", 10000, 1000000, 50000)
total_savings = st.number_input("Total Savings ($)", 0, 1000000, 10000)
total_debt = st.number_input("Total Debt ($)", 0, 500000, 20000)
risk_tolerance = st.slider("Risk Tolerance (0=Low, 1=High)", 0.0, 1.0, 0.5)
credit_score = st.slider("Credit Score", 300, 850, 700)

# Dynamic goal input
st.header("Enter Your Financial Goals")
num_goals = st.number_input("Number of Goals", 1, 10, 1)  # Allow 1-10 goals, default to 1
goals = []
categories = ['retirement', 'housing', 'education', 'vehicle', 'travel', 'business']

for i in range(num_goals):
    st.subheader(f"Goal {i+1}")
    category = st.selectbox(f"Category {i+1}", categories, key=f"cat_{i}")
    target_amount = st.number_input(f"Target Amount ($) {i+1}", 1000, 1000000, 50000, key=f"target_{i}")
    timeline_months = st.selectbox(f"Timeline (Months) {i+1}", [6, 12, 24, 36, 60, 120], key=f"time_{i}")
    current_progress = st.number_input(f"Current Progress ($) {i+1}", 0, target_amount, 0, key=f"progress_{i}")
    monthly_contribution = st.number_input(f"Monthly Contribution ($) {i+1}", 0, int(annual_income / 12 * 0.3), 0, key=f"contrib_{i}")
    if monthly_contribution > annual_income / 12:
        st.error("Monthly contribution exceeds 1/12 of annual income. Please adjust.")
        monthly_contribution = 0  # Reset to avoid invalid input
    goals.append({
        'category': category,
        'target_amount': target_amount,
        'timeline_months': timeline_months,
        'current_progress': current_progress,
        'monthly_contribution': monthly_contribution
    })

# Economic indicators
st.header("Economic Conditions")
inflation_rate = st.slider("Inflation Rate (%)", 1.0, 9.0, 3.0)
interest_rate = st.slider("Interest Rate (%)", 1.5, 12.0, 4.0)
market_sentiment = st.slider("Market Sentiment (0=Low, 1=High)", 0.0, 1.0, 0.5)

if st.button("Prioritize Goals"):
    data = []
    for i, goal in enumerate(goals):
        progress_ratio = goal['current_progress'] / goal['target_amount'] if goal['target_amount'] > 0 else 0
        contribution_to_feasibility = goal['monthly_contribution'] / (goal['target_amount'] / goal['timeline_months']) if goal['timeline_months'] > 0 else 0
        row = {
            'age': age,
            'annual_income': annual_income,
            'total_savings': total_savings,
            'total_debt': total_debt,
            'debt_to_income_ratio': total_debt / annual_income if annual_income > 0 else 0,
            'risk_tolerance_score': risk_tolerance,
            'credit_score': credit_score,
            'target_amount': goal['target_amount'],
            'current_progress': goal['current_progress'],
            'timeline_months': goal['timeline_months'],
            'goal_feasibility': goal['target_amount'] / goal['timeline_months'] if goal['timeline_months'] > 0 else 0,
            'monthly_contribution': goal['monthly_contribution'],
            'progress_ratio': progress_ratio,
            'contribution_to_feasibility': contribution_to_feasibility,
            'inflation_rate': inflation_rate,
            'interest_rate': interest_rate,
            'market_sentiment': market_sentiment,
            'category_business': 0,
            'category_education': 0,
            'category_housing': 0,
            'category_retirement': 0,
            'category_travel': 0,
            'category_vehicle': 0
        }
        if goal['category'] == 'business': row['category_business'] = 1
        elif goal['category'] == 'education': row['category_education'] = 1
        elif goal['category'] == 'housing': row['category_housing'] = 1
        elif goal['category'] == 'retirement': row['category_retirement'] = 1
        elif goal['category'] == 'travel': row['category_travel'] = 1
        elif goal['category'] == 'vehicle': row['category_vehicle'] = 1
        data.append(row)

    input_df = pd.DataFrame(data)
    input_df['category'] = [goal['category'] for goal in goals]  # Add category here
    input_df_original = input_df.copy()  # Copy after adding category
    input_df = input_df[expected_features]

    st.write("Input DataFrame (Original Values):", input_df_original)

    # Apply scaler transformation and predict
    input_df_scaled = input_df.copy()  # Separate scaled DataFrame
    input_df_scaled[expected_features] = scaler.transform(input_df[expected_features])
    input_df['predicted_priority'] = model.predict(input_df_scaled)

    input_df['adjusted_priority'] = input_df.apply(
        lambda row: min(1.0, row['predicted_priority'] * (1 + min(1.0, row['monthly_contribution'] / (row['target_amount'] / row['timeline_months'])) * 1 + (1 / row['timeline_months']) * 2) * max(0.5, 1 - (row['target_amount'] / (row['monthly_contribution'] * row['timeline_months'] if row['monthly_contribution'] * row['timeline_months'] > 0 else 1))))
        if row['target_amount'] > 0 and row['timeline_months'] > 0 else row['predicted_priority'], axis=1)

    def recommend_action(score):
        if score > 0.7:
            return "Focus on this goal"
        elif score > 0.4:
            return "Maintain steady progress"
        else:
            return "Consider delaying or adjusting"

    input_df['recommended_action'] = input_df['adjusted_priority'].apply(recommend_action)

    # Use original values for display
    result_df = input_df_original[['category', 'target_amount', 'timeline_months', 'monthly_contribution']].copy()
    result_df['adjusted_priority'] = input_df['adjusted_priority']
    result_df['recommended_action'] = input_df['recommended_action']
    result_df = result_df.sort_values('adjusted_priority', ascending=False)
    st.header("Prioritized Financial Goals")
    st.dataframe(result_df.style.format({
        'target_amount': '${:,.2f}',
        'monthly_contribution': '${:,.2f}',
        'timeline_months': '{:.0f}',
        'adjusted_priority': '{:.2f}'
    }))

    if st.button("Download Results"):
        result_df.to_csv('user_goals.csv', index=False)
        with open('user_goals.csv', 'rb') as f:
            st.download_button("Download CSV", data=f, file_name='user_goals.csv', mime='text/csv')

    # Chart section
    st.subheader(f"Goal Prioritization Chart ({len(result_df)} Goals)")
    chart_type = st.selectbox("Select Chart Type", ["bar", "pie"], index=0)

    # Prepare chart data
    categories = result_df['category'].tolist()
    priorities = result_df['adjusted_priority'].tolist()
    if not categories or not priorities:
        st.error("No goals to display. Please add at least one goal.")
    else:
        num_colors = min(len(result_df), 6)
        colors = ["#4CAF50", "#2196F3", "#F44336", "#FFC107", "#9C27B0", "#FF9800"][:num_colors]
        border_colors = ["#388E3C", "#1976D2", "#D32F2F", "#FFA000", "#7B1FA2", "#F57C00"][:num_colors]

        if chart_type == "bar":
            chart_data = {
                "xAxis": {"type": "category", "data": categories},
                "yAxis": {"type": "value", "max": 1.0},
                "series": [{
                    "name": "Priority Score",
                    "type": "bar",
                    "data": priorities,
                    "itemStyle": {"color": colors[0] if len(priorities) > 0 else "#4CAF50"}
                }],
                "title": {"text": f"Prioritized Financial Goals for {len(result_df)} Goals"}
            }
        elif chart_type == "pie":
            chart_data = {
                "series": [{
                    "name": "Priority Distribution",
                    "type": "pie",
                    "radius": "70%",
                    "data": [{"value": p, "name": c} for c, p in zip(categories, priorities)],
                    "itemStyle": {"color": colors[0] if len(priorities) > 0 else "#4CAF50"}
                }],
                "title": {"text": f"Priority Distribution for {len(result_df)} Goals"}
            }

        # Debug: Verify data
        # st.write("Debug - Chart Data:", {"labels": categories, "data": priorities})

        try:
            st_echarts(options=chart_data, height="400px")
        except Exception as e:
            st.error(f"Chart error: {str(e)}. Please check the data or try again.")
            st.write("Debug - Result DF:", result_df[['category', 'adjusted_priority']].to_dict())

# Update streamlit-echarts if needed
# st.write("Ensuring streamlit-echarts is up-to-date...")
# st.write("If chart doesn't render, restart the app after running: !pip install --upgrade streamlit-echarts")

Overwriting app.py


In [14]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
!pip install pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(public_url)

ERROR:pyngrok.process.ngrok:t=2025-07-21T08:57:27+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-21T08:57:27+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-21T08:57:27+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [15]:
!curl https://localtunnel.me/mytunnelpassword

35.193.81.20

In [18]:
# Run Streamlit in the background
!streamlit run app.py &>/dev/null&

# Start localtunnel on port 8501 (default Streamlit port)
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://metal-paths-lose.loca.lt
^C


In [13]:
!pkill -f streamlit
!pkill -f localtunnel

In [8]:
!pip install streamlit-echarts

In [ ]:
!pip install --upgrade streamlit-echarts